In [1]:
#'##########################################################
#	  	      MCMC GLMM
# 	a sript for  the project
#	DEMOGRAPHIC BUFFERING CONTINUUM in PLANTS AND ANIMALS
#			 by Gabriel Santos
# 	contact by ssantos.gabriel@gmail.com
#			    05 March 2025
#' ---------------------------------------------------------
# MCMCglmms run separately so we can run it
# with intermediary data produced
#'##########################################################
set.seed(1)

rm(list=ls())

# Define necessary packages
need_pkgs <- c("tidyverse", "plotMCMC", "mcmcr", "MuMIn", "MCMCglmm","MuMIn")

exist_pckgs<-exist_pckgs <- installed.packages()[, "Package"]

if (any(!need_pkgs %in% exist_pckgs)) {   # Check for inexisting packages and install them
  install.packages(need_pkgs[!need_pkgs %in% exist_pckgs])
}




In [2]:
# load non-existing packages
lapply(need_pkgs, require, character.only = TRUE)

rm(list=ls())

Loading required package: tidyverse
── Attaching core tidyverse packages ───────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ─────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the ]8;;http://conflicted.r-lib.org/conflicted package]8;; to force all conflicts to become errors
Loading required package: plotMCMC
Loading required package: mcmcr
Registered S3 method overwritten by 'mcmcr':
  method         from 
  as.mcmc.nlists nlist
Loading required package: MuMIn
Loading required package: MCMCglmm
Loading required package: Matrix

Attaching package: ‘Matrix’

The following objects are masked from ‘package:tidyr’:

    expand, pack, unpack

Loading required package: coda
Loadin

Warning messages:
1: package ‘plotMCMC’ was built under R version 4.4.3 
2: package ‘mcmcr’ was built under R version 4.4.3 
3: package ‘MCMCglmm’ was built under R version 4.4.2 


In [3]:

#'===========================================================
# ----- LOAD data ready for GLMM analyses ----
# GLMM data contains:
#  - Data model
#  - subtree_Animals - Phylogenetic tree for Animals ready for analysis
#  - subtree_Animals - Phylogenetic tree for Plants ready for analysis
#'===========================================================
# USE THIS IF RUNNING IN GOOGLE COLAB
# Download data In Google colab
#dir.create(file.path("Data"), showWarnings = FALSE) # Will return warning message if folder already exists
#GLMMdata_link<-"https://github.com/Ecosantos/Demogbuff-pops/raw/refs/heads/incorporating-MCMCGlmm/Data/GLMMdata.Rdata"
#download.file(GLMMdata_link, "Data/GLMMdata.Rdata", mode = "wb")
#'===========================================================

load("Data/GLMMdata.Rdata")

#'===========================================================
# ---- Define GLMM SETTINGS ----
#'===========================================================

fixEffect<-fixEffect<-"~LHPC.1 * LHPC.2 + ClimPC.1 * ClimPC.2"
InterestingVars<-c("Survival","Growth","Shrinking","Reproduction","Clonality","Buffmx","Cumulative")

traits<-traits_glmm<- unique (grep(paste(InterestingVars,collapse="|"),
                                   colnames(data_model), value=TRUE))

#'-----------------------------------------------------------
# ---- Define priors ----
#'-----------------------------------------------------------

prior_phylo<-list(G=list(G1=list(V=1,nu=0.02)),
                  R=list(V=1,nu=0.02))

nitt=100000; #nitt=1000
burnin=1000; #burnin=100
thin=10

glmmScale<-"FALSE"

In [4]:
#'===========================================================
#	PHYLOGENETIC MCMC GLMM
#'===========================================================
MCMCglmm_phylo_animals<-MCMCglmm_phylo_plants<-NULL

#Animals	phylo
for(i in 1:length(traits)){
print(paste("Running model:",traits[i],"~",fixEffect))
MCMCglmm_phylo_animals[[i]]<-MCMCglmm(formula(paste0(traits[i], fixEffect)),
    random=~phylo,family="gaussian",
		ginverse=list(phylo=inverseA(subtree_Animals,nodes="TIPS",scale=TRUE)$Ainv),
				prior=prior_phylo,data=subset(data_model,Kingdom=="Animalia"),
   						nitt=nitt,burnin=burnin,thin=thin,singular.ok=TRUE, scale = glmmScale)

names(MCMCglmm_phylo_animals)[i]<-traits[[i]]
}


#Plants
for(i in 1:length(traits)){
print(paste("Running model:",traits[i],"~",fixEffect))
MCMCglmm_phylo_plants[[i]]<-MCMCglmm(formula(paste0(traits[i], fixEffect)),
    random=~phylo,family="gaussian",
		ginverse=list(phylo=inverseA(subtree_Plants,nodes="TIPS",scale=TRUE)$Ainv),
				prior=prior_phylo,data=subset(data_model,Kingdom=="Plantae"),
   						nitt=nitt,burnin=burnin,thin=thin,singular.ok=TRUE, scale = glmmScale)

names(MCMCglmm_phylo_plants)[i]<-traits[[i]]
}

#'===========================================================
#	SIMPLE MCMC GLMM
#'===========================================================
MCMCglmm_simple_animals<-MCMCglmm_simple_plants<-NULL

#Animals
for(i in 1:length(traits)){
print(paste("Running model:",traits[i],"~",fixEffect))
MCMCglmm_simple_animals[[i]]<-MCMCglmm(formula(paste0(traits[i], fixEffect)),
    family="gaussian",data=subset(data_model,Kingdom=="Animalia"),
   			nitt=nitt,burnin=burnin,thin=thin,singular.ok=TRUE, scale = glmmScale)

names(MCMCglmm_simple_animals)[i]<-traits[[i]]
}

#Plants
for(i in 1:length(traits)){
print(paste("Running model:",traits[i],"~",fixEffect))
MCMCglmm_simple_plants[[i]]<-MCMCglmm(formula(paste0(traits[i], fixEffect)),
   family="gaussian",data=subset(data_model,Kingdom=="Plantae"),
   			nitt=nitt,burnin=burnin,thin=thin,singular.ok=TRUE, scale = glmmScale)

names(MCMCglmm_simple_plants)[i]<-traits[[i]]
}







[1] "Running model: Reproduction_SigElas ~ ~LHPC.1 * LHPC.2 + ClimPC.1 * ClimPC.2"

                       MCMC iteration = 0

                       MCMC iteration = 1000

                       MCMC iteration = 2000

                       MCMC iteration = 3000

                       MCMC iteration = 4000

                       MCMC iteration = 5000

                       MCMC iteration = 6000

                       MCMC iteration = 7000

                       MCMC iteration = 8000

                       MCMC iteration = 9000

                       MCMC iteration = 10000

                       MCMC iteration = 11000

                       MCMC iteration = 12000

                       MCMC iteration = 13000

                       MCMC iteration = 14000

                       MCMC iteration = 15000

                       MCMC iteration = 16000

                       MCMC iteration = 17000

                       MCMC iteration = 18000

                       MCMC iteratio

In [5]:
glmmOUT<-list(Simple_models=
       list(Plants = MCMCglmm_simple_plants,Animals = MCMCglmm_simple_animals),
     Phylogenetic_models=
       list(Plants = MCMCglmm_phylo_plants,Animals = MCMCglmm_phylo_animals)
     )

#saveRDS(glmmOUT,"Data/MCMCglmm_output.rds")